In [69]:
import os, sys
import django
from datetime import datetime
sys.path.append('../') # add path to project root dir
import pandas as pd
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [70]:
SubjectScheduleHistory.objects.first().__dict__

{'_state': <django.db.models.base.ModelState at 0x14f661350>,
 'created': datetime.datetime(2022, 6, 21, 6, 33, 15, 582494, tzinfo=<UTC>),
 'modified': datetime.datetime(2022, 6, 21, 6, 33, 15, 582514, tzinfo=<UTC>),
 'user_created': '',
 'user_modified': '',
 'hostname_created': 'flourish',
 'hostname_modified': 'flourish',
 'revision': '',
 'device_created': '99',
 'device_modified': '99',
 'id': UUID('00260ebb-b306-4eaa-80ee-d3b1a5b6a228'),
 'subject_identifier': 'B142-040990704-4',
 'visit_schedule_name': 'a1_visit_schedule1',
 'schedule_name': 'a_birth1_schedule1',
 'onschedule_model': 'flourish_caregiver.onschedulecohortabirth',
 'offschedule_model': 'flourish_caregiver.caregiveroffschedule',
 'onschedule_datetime': datetime.datetime(2022, 6, 18, 0, 0, tzinfo=<UTC>),
 'offschedule_datetime': None,
 'schedule_status': 'onschedule'}

In [71]:
onschedule_models = set(SubjectScheduleHistory.objects.filter(
    onschedule_model__istartswith='flourish_caregiver'
).values_list('onschedule_model', flat=True))

onschedule_models

{'flourish_caregiver.onschedulecohortaantenatal',
 'flourish_caregiver.onschedulecohortabirth',
 'flourish_caregiver.onschedulecohortaenrollment',
 'flourish_caregiver.onschedulecohortafu',
 'flourish_caregiver.onschedulecohortafuquarterly',
 'flourish_caregiver.onschedulecohortaquarterly',
 'flourish_caregiver.onschedulecohortatb2months',
 'flourish_caregiver.onschedulecohortatb6months',
 'flourish_caregiver.onschedulecohortbenrollment',
 'flourish_caregiver.onschedulecohortbfu',
 'flourish_caregiver.onschedulecohortbfuquarterly',
 'flourish_caregiver.onschedulecohortbquarterly',
 'flourish_caregiver.onschedulecohortbsec',
 'flourish_caregiver.onschedulecohortbsecquart',
 'flourish_caregiver.onschedulecohortcenrollment',
 'flourish_caregiver.onschedulecohortcfu',
 'flourish_caregiver.onschedulecohortcfuquarterly',
 'flourish_caregiver.onschedulecohortcquarterly',
 'flourish_caregiver.onschedulecohortcsec',
 'flourish_caregiver.onschedulecohortcsecquart'}

In [72]:
child_subject_identifiers = set(CaregiverChildConsent.objects.values_list('subject_identifier', flat=True))

child_subject_identifiers

{'B142-040990528-7-60',
 'B142-040990624-4-10',
 'B142-040990073-4-10',
 'B142-040990809-1-10',
 'B142-040990812-5-10',
 'C142-040990091-6-10',
 'B142-040990020-5-10',
 'B142-040990459-5-10',
 'B142-040990925-5-10',
 'B142-040990111-2-10',
 'B142-040990375-3-10',
 'B142-040990786-1-60',
 'B142-040990977-6-10',
 'B142-040991087-3-10',
 'B142-040991035-2-10',
 'B142-040990075-9-10',
 'B142-040990386-0-10',
 'B142-040990692-1-10',
 'B142-040991110-3-10',
 'B142-040990401-7-10',
 'B142-040990285-4-10',
 'B142-040990647-5-10',
 'B142-040990306-8-10',
 'C142-040990629-3-10',
 'B142-040991135-0-10',
 'B142-040990901-6-10',
 'B142-040990632-7-10',
 'B142-040990899-2-10',
 'B142-040990435-5-10',
 'B142-040991071-7-10',
 'B142-040990796-0-10',
 'B142-040990115-3-10',
 'B142-040990725-9-10',
 'B142-040990858-8-10',
 'B142-040990455-3-10',
 'B142-040990873-7-10',
 'B142-040990339-9-10',
 'B142-040990121-1-10',
 'B142-040991099-8-10',
 'B142-040990504-8-10',
 'B142-040990141-9-10',
 'B142-040990531

In [78]:
duplicates_list = []

for subject_identifier in child_subject_identifiers:
    for model in onschedule_models:
        model_cls = django_apps.get_model(model)
        model_objs = model_cls.objects.filter(
            child_subject_identifier=subject_identifier)
        
        if model_objs.count() > 1:
            temp = dict()
            temp['child_subject_identifier'] = subject_identifier
            temp['onschedule_model'] = model
            temp['caregiver_subject_identifier'] =  model_objs.latest('onschedule_datetime').subject_identifier
            temp['schedule_names'] = set(model_objs.values_list('schedule_name', flat=True))
            duplicates_list.append(temp)
            
duplicates_list or 'No duplicates'


'No duplicates'

In [76]:
from edc_appointment.constants import COMPLETE_APPT, IN_PROGRESS_APPT

for duplicate in duplicates_list:
    
    child_subject_identifier = duplicate['child_subject_identifier']
    caregiver_subject_identifier = duplicate['caregiver_subject_identifier']
    schedule_names = duplicate['schedule_names']
    onschedule_name = duplicate['onschedule_model']
    
    onschedule_cls = django_apps.get_model(duplicate['onschedule_model'])
    
    appts = CaregiverAppointment.objects.filter(
            appt_status__in=[COMPLETE_APPT, IN_PROGRESS_APPT],
            schedule_name__in = schedule_names,
            subject_identifier = caregiver_subject_identifier)
    
    appt_schedule_names = appts.values_list('schedule_name', flat=True)    
    
    try:
    
        schedule_obj = onschedule_cls.objects.filter(
            subject_identifier = caregiver_subject_identifier,
            child_subject_identifier = child_subject_identifier,
            schedule_name__in = schedule_names
        ).exclude(schedule_name__in=appt_schedule_names).latest('onschedule_datetime')
        
        subject_schedule_history_obj = SubjectScheduleHistory.objects.get(
            subject_identifier = caregiver_subject_identifier,
            schedule_name = schedule_obj.schedule_name,
            onschedule_model = onschedule_name,
        )
        
    except onschedule_cls.DoesNotExist:
        pass
    else:
        print(subject_identifier)
        schedule_obj.delete()
        subject_schedule_history_obj.delete()
        
        
        CaregiverAppointment.objects.filter(
        subject_identifier = caregiver_subject_identifier,
        schedule_name = schedule_obj.schedule_name).delete()

        
    